## Rearrange Dataset for Dataloader

In [20]:
!python restructure_data.py

Transferred 200 images and 200 targets in train set
Transferred 100 images and 100 targets in test set


## Test dataset class

In [2]:
from dataset import *

data_path = 'data/'
data = SegmentationDataSet(data_path, train=True)

In [2]:
data.__len__()

201

In [3]:
x, y = data.__getitem__(3)

NiftiImageIO (0x34fa4b0): data/train/images/patient041_frame01.nii.gz has unexpected scales in sform

NiftiImageIO (0x34fa4b0): data/train/images/patient041_frame01.nii.gz has unexpected scales in sform

NiftiImageIO (0x3cda790): data/train/images/patient041_frame01.nii.gz has unexpected scales in sform

NiftiImageIO (0x3cda790): data/train/images/patient041_frame01.nii.gz has unexpected scales in sform



In [4]:
x.shape

(6, 224, 154)

In [5]:
y.shape

(6, 224, 154)

In [6]:
from transformations import *

x = resize_image(x)

In [8]:
x.shape

(10, 256, 256)

## Test dataset class with transform

In [3]:
from torchvision import transforms
from transformations import *

train_transform = transforms.Compose([transforms.Lambda(normalise), 
                                     transforms.Lambda(resize_image),
                                     transforms.Lambda(np_to_tensor)])

data_path = 'data/'
data_transform = SegmentationDataSet(data_path, train=True, transform=train_transform)

In [4]:
x,y = data_transform.__getitem__(3)

In [5]:
x.shape

torch.Size([10, 256, 256])

In [6]:
y.shape

torch.Size([10, 256, 256])

In [7]:
from torch.utils.data import DataLoader
dataloader_training = DataLoader(dataset=data_transform,
                                 batch_size=4,
                                 shuffle=True)

In [8]:
x, y = next(iter(dataloader_training))

In [9]:
x.shape

torch.Size([4, 10, 256, 256])

In [19]:
y.shape

torch.Size([4, 10, 256, 256])

In [10]:
x[None, :].shape


torch.Size([1, 4, 10, 256, 256])

In [22]:
!python main2.py

Train Set:  160
Validation Set:  41
Test Set:  100


In [27]:
sitk.ProcessObject_SetGlobalWarningDisplay(False)

In [32]:
!python main2.py

Train Set:  201
Test Set:  100
Training:   0%|                                          | 0/13 [00:00<?, ?it/s]HDF5-DIAG: Error detected in HDF5 (1.12.1) thread 0:
  #000: /tmp/SimpleITK-build/ITK/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5Fdeprec.c line 156 in itk_H5Fis_hdf5(): unable to determine if file is accessible as HDF5
    major: File accessibility
    minor: Not an HDF5 file
  #001: /tmp/SimpleITK-build/ITK/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5VLcallback.c line 3769 in itk_H5VL_file_specific(): file specific failed
    major: Virtual Object Layer
    minor: Can't operate on object
  #002: /tmp/SimpleITK-build/ITK/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5VLcallback.c line 3699 in H5VL__file_specific(): file specific failed
    major: Virtual Object Layer
    minor: Can't operate on object
  #003: /tmp/SimpleITK-build/ITK/Modules/ThirdParty/HDF5/src/itkhdf5/src/H5VLnative_file.c line 384 in itk_H5VL__native_file_specific(): error in HDF5 file check
    major: File accessib

In [17]:
import torch
from unet import UNet
model = UNet(in_channels=1,
             out_channels=4,
             n_blocks=3,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=3)

x = torch.randn(size=(1, 1, 40, 256, 256), dtype=torch.float32)
with torch.no_grad():
    out = model(x)

print(f'Out: {out.shape}')

Out: torch.Size([1, 4, 40, 256, 256])


In [15]:
from torchsummary import summary
summary = summary(model, (1, 256, 256))

RuntimeError: Given groups=1, weight of size [32, 1, 3, 3, 3], expected input[1, 2, 1, 256, 256] to have 1 channels, but got 2 channels instead

In [6]:
!pip install torchsummary

Defaulting to user installation because normal site-packages is not writeable
  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


In [16]:
shape = 40

def compute_max_depth(shape, max_depth=10, print_out=True):
    shapes = []
    shapes.append(shape)
    for level in range(1, max_depth):
        if shape % 2 ** level == 0 and shape / 2 ** level > 1:
            shapes.append(shape / 2 ** level)
            if print_out:
                print(f'Level {level}: {shape / 2 ** level}')
        else:
            if print_out:
                print(f'Max-level: {level - 1}')
            break

    return shapes

out = compute_max_depth(shape, print_out=True, max_depth=10)

Level 1: 20.0
Level 2: 10.0
Level 3: 5.0
Max-level: 3
